In [13]:

from netCDF4 import Dataset, num2date
import pandas as pd
import numpy as np
import xarray as xr
nc_file = 'gennaio.nc'
#ds = xr.open_dataset('a492415c2645b30009ac40c242fa6287.nc', engine='cfgrib')
ds = Dataset(nc_file, mode='r')

In [4]:
from netCDF4 import Dataset
import numpy as np
import pandas as pd

# Try opening the file
ds = Dataset("gennaio")

# List variables
print(ds.variables.keys())

OSError: [Errno -51] NetCDF: Unknown file format: 'gennaio'

In [32]:
import cfgrib

ds = cfgrib.open_dataset("era5land_2022_01.nc")
print(ds)

ValueError: unrecognized engine 'cfgrib' must be one of your download engines: ['scipy', 'store']. To install additional dependencies, see:
https://docs.xarray.dev/en/stable/user-guide/io.html 
https://docs.xarray.dev/en/stable/getting-started-guide/installing.html

In [23]:
import xarray.backends

print(xarray.backends.list_engines())

{'scipy': <ScipyBackendEntrypoint>
  Open netCDF files (.nc, .nc4, .cdf and .gz) using scipy in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.ScipyBackendEntrypoint.html, 'store': <StoreBackendEntrypoint>
  Open AbstractDataStore instances in Xarray
  Learn more at https://docs.xarray.dev/en/stable/generated/xarray.backends.StoreBackendEntrypoint.html}


In [ ]:
#print(ds)
temperature = ds.variables['t2m']
precipitations = ds.variables['tp']*1000
#pressure = ds.
print(temperature)



<class 'netCDF4.Variable'>
float32 t2m(valid_time, latitude, longitude)
    _FillValue: nan
    GRIB_paramId: 167
    GRIB_dataType: fc
    GRIB_numberOfPoints: 1
    GRIB_typeOfLevel: surface
    GRIB_stepUnits: 1
    GRIB_stepType: instant
    GRIB_gridType: regular_ll
    GRIB_uvRelativeToGrid: 0
    GRIB_NV: 0
    GRIB_Nx: 1
    GRIB_Ny: 1
    GRIB_cfName: unknown
    GRIB_cfVarName: t2m
    GRIB_gridDefinitionDescription: Latitude/Longitude Grid
    GRIB_iDirectionIncrementInDegrees: 0.1
    GRIB_iScansNegatively: 0
    GRIB_jDirectionIncrementInDegrees: 0.1
    GRIB_jPointsAreConsecutive: 0
    GRIB_jScansPositively: 0
    GRIB_latitudeOfFirstGridPointInDegrees: 44.34
    GRIB_latitudeOfLastGridPointInDegrees: 44.34
    GRIB_longitudeOfFirstGridPointInDegrees: 12.25
    GRIB_longitudeOfLastGridPointInDegrees: 12.25
    GRIB_missingValue: 3.4028234663852886e+38
    GRIB_name: 2 metre temperature
    GRIB_shortName: 2t
    GRIB_totalNumber: 0
    GRIB_units: K
    long_name: 2 metr

In [12]:
temperature = ds.variables['t2m'][:]       # shape: (time, lat, lon)
time = ds.variables['valid_time']
lat = ds.variables['latitude'][:]
lon = ds.variables['longitude'][:]

# Convert time units to datetime objects
time_units = time.units
time_calendar = time.calendar if hasattr(time, 'calendar') else 'standard'
dates = num2date(time[:], units=time_units, calendar=time_calendar)

# Reshape data: (time, lat, lon) -> flat table
times, lats, lons = np.meshgrid(dates, lat, lon, indexing='ij')
data_flat = {
    'time': times.ravel(),
    'latitude': lats.ravel(),
    'longitude': lons.ravel(),
    't2m': temperature.ravel()
}

# Create DataFrame
df = pd.DataFrame(data_flat)

# Optional: Convert temperature from Kelvin to Celsius
df['t2m'] = df['t2m'] - 273.15

In [13]:
import cdsapi
import os

# Initialize client
client = cdsapi.Client()

# Year you want to download
year = 2022

# Define variables and fixed request parts
dataset = "reanalysis-era5-land"
variables = [
    "2m_dewpoint_temperature",
    "2m_temperature",
    "surface_pressure",
    "total_precipitation"
]
times = [f"{hour:02d}:00" for hour in range(24)]  # All 24 hours

# Latitude/Longitude bounds [N, W, S, E]
area = [44.35, 12.25, 44.34, 12.26]

# Create output directory if it doesn't exist
output_dir = f"era5_{year}"
os.makedirs(output_dir, exist_ok=True)

# Loop over each month
for month in range(1, 13):
    month_str = f"{month:02d}"
    
    # Determine days in the month
    if month_str in ['01', '03', '05', '07', '08', '10', '12']:
        days = [f"{day:02d}" for day in range(1, 32)]
    elif month_str in ['04', '06', '09', '11']:
        days = [f"{day:02d}" for day in range(1, 31)]
    else:  # February (leap year check)
        days = [f"{day:02d}" for day in range(1, 30 if year % 4 == 0 and (year % 100 != 0 or year % 400 == 0) else 29)]

    # Prepare request
    request = {
        "variable": variables,
        "year": str(year),
        "month": month_str,
        "day": days,
        "time": times,
        "format": "netcdf",
        "area": area
    }

    # Output file name
    filename = os.path.join(output_dir, f"era5land_{year}_{month_str}.nc")

    # Download
    print(f"Downloading {filename}...")
    client.retrieve(dataset, request).download(filename)

print("✅ Download completed for all months.")

2025-06-03 15:58:53,113 INFO [2024-09-26T00:00:00] Watch our [Forum](https://forum.ecmwf.int/) for Announcements, news and other discussed topics.


2025-06-03 15:58:53,481 INFO Request ID is d466cb85-db6c-4fb1-b010-8a45404767b5
2025-06-03 15:58:53,586 INFO status has been updated to accepted
2025-06-03 15:59:07,222 INFO status has been updated to running
2025-06-03 16:07:17,819 INFO status has been updated to successful


2025-06-03 16:07:18,724 INFO Request ID is 533dfc9d-8961-456b-aec6-931820340f87
2025-06-03 16:07:18,810 INFO status has been updated to accepted
2025-06-03 16:07:32,663 INFO status has been updated to running
2025-06-03 16:19:38,820 INFO status has been updated to successful


2025-06-03 16:19:39,748 INFO Request ID is c7c559aa-338c-430f-8525-79b55751e2b4
2025-06-03 16:19:39,820 INFO status has been updated to accepted
2025-06-03 16:19:48,414 INFO status has been updated to running
2025-06-03 16:28:04,295 INFO status has been updated to successful


2025-06-03 16:28:05,754 INFO Request ID is 292dc184-8530-41b7-9dcd-c80f0ebdf831
2025-06-03 16:28:05,820 INFO status has been updated to accepted
2025-06-03 16:28:27,016 INFO status has been updated to running
2025-06-03 16:36:24,790 INFO status has been updated to successful


...

2025-06-03 17:11:53,482 INFO Request ID is b07b6718-0c13-432e-840a-78a0b5a126f4
2025-06-03 17:11:53,558 INFO status has been updated to accepted
2025-06-03 17:12:07,490 INFO status has been updated to running
2025-06-03 17:22:13,492 INFO status has been updated to successful


2025-06-03 17:22:14,454 INFO Request ID is dc4f490d-987f-4160-89eb-a8242e3ffb73
2025-06-03 17:22:14,524 INFO status has been updated to accepted
2025-06-03 17:23:30,370 INFO status has been updated to running
2025-06-03 17:32:35,000 INFO status has been updated to successful


2025-06-03 17:32:35,915 INFO Request ID is 6f58bdbd-0ef5-4eb4-bf08-89c3d763bc7c
2025-06-03 17:32:36,273 INFO status has been updated to accepted
2025-06-03 17:32:41,333 INFO status has been updated to running
2025-06-03 17:42:55,705 INFO status has been updated to successful
2025-06-03 17:42:56,539 INFO Request ID is 7f878bd1-5dea-448a-b53b-21f34974aa6d        


2025-06-03 17:42:56,648 INFO status has been updated to accepted
2025-06-03 17:43:10,254 INFO status has been updated to running
2025-06-03 17:55:16,205 INFO status has been updated to successful
                                                                                       

✅ Download completed for all months.


In [ ]:
import numpy as np

def compute_relative_humidity(temp_c, dewpoint_c):
    """
    Calculate relative humidity (%) from temperature and dew point (in Celsius).
    Uses the Magnus formula.
    """
    a, b = 17.625, 243.04
    rh = 100 * (np.exp((a * dewpoint_c) / (b + dewpoint_c)) /
                np.exp((a * temp_c) / (b + temp_c)))
    return np.clip(rh, 0, 100)  # Ensure RH is within 0-100%

In [ ]:
temp_c = temperature - 273.15
dewpoint_c = dewpoint_k - 273.15
rh = compute_relative_humidity(temp_c, dewpoint_c)